In [61]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Programs\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [62]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Programs\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [63]:
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [65]:
address = 'Times Square, NY'

geolocator = Nominatim(user_agent="gazetteer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Times Square is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Times Square is 40.757280550000004, -73.98585503545917.


In [89]:
radius = 1000
search_query = 'restaurant'

In [90]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

In [91]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,462a6065f964a520d9451fe3,1515 Broadway,US,New York,United States,at W 45th St,131,"[1515 Broadway (at W 45th St), New York, NY 10...","[{'label': 'display', 'lat': 40.75834934354621...",40.758349,-73.986513,Theater District,10036,NY,Junior's Restaurant & Bakery,v-1612500775,77465459
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a5209ee81ee3,200 W 44th St,US,New York,United States,btwn Broadway & 8th Ave,81,"[200 W 44th St (btwn Broadway & 8th Ave), New ...","[{'label': 'display', 'lat': 40.7574973, 'lng'...",40.757497,-73.986779,NaN,10036,NY,Carmine’s Italian Restaurant,v-1612500775,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",2078043,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/the-view-restaur...,False,3fd66200f964a520bfea1ee3,1535 Broadway,US,New York,United States,47th & 48th Fl,161,"[1535 Broadway (47th & 48th Fl), New York, NY ...","[{'label': 'display', 'lat': 40.75869, 'lng': ...",40.758690,-73.986285,NaN,10036,NY,The View Restaurant & Lounge,v-1612500775,60159912
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e8cd404775bde318a4d6454,156 W 44th St,US,New York,United States,at 7th Ave,62,"[156 W 44th St (at 7th Ave), New York, NY 1003...","[{'label': 'display', 'lat': 40.75701783963928...",40.757018,-73.985199,NaN,10036,NY,O'Donoghues Pub & Restaurant,v-1612500775,NaN
4,"[{'id': '52e81612bcbc57f1066b7a06', 'name': 'I...",801912,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/connollys-irish-...,False,4a6a25c0f964a520b7cc1fe3,121 W 45th St,US,New York,United States,at 6th Ave,192,"[121 W 45th St (at 6th Ave), New York, NY 1003...","[{'label': 'display', 'lat': 40.7573681, 'lng'...",40.757368,-73.983580,NaN,10036,NY,Connolly's Pub & Restaurant,v-1612500775,NaN


In [92]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

del dataframe_filtered['neighborhood']

dataframe_filtered.size

450

In [93]:
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Junior's Restaurant & Bakery,American Restaurant,1515 Broadway,US,New York,United States,at W 45th St,131,"[1515 Broadway (at W 45th St), New York, NY 10...","[{'label': 'display', 'lat': 40.75834934354621...",40.758349,-73.986513,10036,NY,462a6065f964a520d9451fe3
1,Carmine’s Italian Restaurant,Italian Restaurant,200 W 44th St,US,New York,United States,btwn Broadway & 8th Ave,81,"[200 W 44th St (btwn Broadway & 8th Ave), New ...","[{'label': 'display', 'lat': 40.7574973, 'lng'...",40.757497,-73.986779,10036,NY,3fd66200f964a5209ee81ee3
2,The View Restaurant & Lounge,Restaurant,1535 Broadway,US,New York,United States,47th & 48th Fl,161,"[1535 Broadway (47th & 48th Fl), New York, NY ...","[{'label': 'display', 'lat': 40.75869, 'lng': ...",40.758690,-73.986285,10036,NY,3fd66200f964a520bfea1ee3
3,O'Donoghues Pub & Restaurant,Bar,156 W 44th St,US,New York,United States,at 7th Ave,62,"[156 W 44th St (at 7th Ave), New York, NY 1003...","[{'label': 'display', 'lat': 40.75701783963928...",40.757018,-73.985199,10036,NY,4e8cd404775bde318a4d6454
4,Connolly's Pub & Restaurant,Irish Pub,121 W 45th St,US,New York,United States,at 6th Ave,192,"[121 W 45th St (at 6th Ave), New York, NY 1003...","[{'label': 'display', 'lat': 40.7573681, 'lng'...",40.757368,-73.983580,10036,NY,4a6a25c0f964a520b7cc1fe3


In [94]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Times Square',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map